# DS 2002 Midterm Project: 'AdventureWorks' Analysis

## Preparing to Construct a New ETL Process
- I downloaded the AdventureWorks SQL file from the DS 2002 Github page, then imported it into MySQL Workbench
### 1.0 Importing Libraries

In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
from sqlalchemy import create_engine, text

### 1.1 Defining Connection Variables

In [2]:
# Connection variables for MySQL
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "athena101"

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

# Connection variables for MongoDB
mongodb_args = {
    "user_name" : "gwb9pe",
    "password" : "lcHYPLt9FbbA3JAW",
    "cluster_name" : "athena",
    "cluster_subnet" : "q7l7m",
    "cluster_location" : "atlas",
    "db_name" : "adventureworks"
}

### 1.2 Defining Connection Functions for Reading and Writing

In [3]:
# Code from Lab 3
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    

# Code from Lab 4
def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

### 1.3 Creating a New Empty Data Warehouse, 'adventureworks_dw'

In [4]:
# Code from Lab 3
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

## Creating Dimension Tables
### 2.0 Creating Date Dimension Table
- I ran the script from Lab 2C to create and populate dim_date
- I noticed that the dates range from 1996-2004, so I adjusted the start date in the script
- Below, I am verifying that dim_date has been created and populated correctly

In [5]:
# Verifying dim_date has been created and populated correctly
sql_dim_date = "SELECT * FROM adventureworks_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.head()

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,19950101,1995-01-01,1995/01/01,01/01/1995,01/01/1995,1,Sunday,1,1,Weekend,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
1,19950102,1995-01-02,1995/01/02,01/02/1995,02/01/1995,2,Monday,2,2,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
2,19950103,1995-01-03,1995/01/03,01/03/1995,03/01/1995,3,Tuesday,3,3,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
3,19950104,1995-01-04,1995/01/04,01/04/1995,04/01/1995,4,Wednesday,4,4,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
4,19950105,1995-01-05,1995/01/05,01/05/1995,05/01/1995,5,Thursday,5,5,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3


### 2.1 Creating Employee Dimension Table
- I am extracting data from the 'employee' table, then the 'contact' table from the MySQL database to create the employee dimension table, 'dim_employees'

In [6]:
# Extracting data from 'employee' table
sql_employees = "SELECT * FROM adventureworks.employee;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head()

,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,Title,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,1972-05-15,M,M,1996-07-31,b'\x00',21,30,b'\x01',b'J\xd0\xe1\xaa7\xc2tI\xb4\xd5\x93RGsw\x18',2004-07-31
1,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,1977-06-03,S,M,1997-02-26,b'\x00',42,41,b'\x01',b'@\x02H\x1b\xc0\x95\x0fA\xa7\x17\xeb)\x94<\x8...,2004-07-31
2,3,509647174,1002,adventure-works\roberto0,12.0,Engineering Manager,1964-12-13,M,M,1997-12-12,b'\x01',2,21,b'\x01',"b',\xfb\xbb\x9b\xbb\xef\x17B\x9a\xb7\xf9v\x892...",2004-07-31
3,4,112457891,1290,adventure-works\rob0,3.0,Senior Tool Designer,1965-01-23,S,M,1998-01-05,b'\x00',48,80,b'\x01',b'UytY\xb8\x87?D\x8e\xd4\xf8\xad:\xfd\xf3\xa9',2004-07-31
4,5,480168528,1009,adventure-works\thierry0,263.0,Tool Designer,1949-08-29,M,M,1998-01-11,b'\x00',9,24,b'\x01',b'qQ\x95\x1ds\xe7\xadO\x83\x82@\xfd\x89\x8d]M',2004-07-31


In [7]:
# Removing unnecessary columns from the table, 'SalariedFlag' and 'CurrentFlag'
df_employees['SalariedFlag'] = df_employees['SalariedFlag'].str[-1]
df_employees['CurrentFlag'] = df_employees['CurrentFlag'].str[-1]

In [8]:
# Extracting data from 'contact' table
sql_contact = "SELECT * FROM adventureworks.contact;"
df_contact = get_dataframe(user_id, pwd, host_name, src_dbname, sql_contact)
df_contact.head()

,ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
0,1,b'\x00',Mr.,Gustavo,None,Achong,None,gustavo0@adventure-works.com,2,398-555-0132,F295DC2A87C7FD93151A2BC232BEAFF1133F95A2,TVGHbhY=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'\xd32\xc1\xd4\xb5\xfc1B\x9d\xd5\x88\x8aT\xbe...,2005-05-16 16:33:33
1,2,b'\x00',Ms.,Catherine,R.,Abel,None,catherine0@adventure-works.com,1,747-555-0171,19712A42FC40F14655499D5058E6A877FE9C367B,rpyd5Tw=,"<AdditionalContactInfo xmlns=""http://schemas.m...","b'R\x05N\xd5&\xc2""L\xaf;v,\xa8T\xcd\xd3'",2005-05-16 16:33:33
2,3,b'\x00',Ms.,Kim,None,Abercrombie,None,kim2@adventure-works.com,0,334-555-0137,CBCD65769648CCBA7693450991E4388894F13723,rrgbG/U=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'H\xdb\xcb\xf7D\x0b\x0eG\x9f7p`Do\xbf\xb9',2005-05-16 16:33:33
3,4,b'\x00',Sr.,Humberto,None,Acevedo,None,humberto0@adventure-works.com,2,599-555-0127,69CF91B5628FA4217743B30C5FE14B5B80A03DB2,F5qyyxs=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'6\xd3AZ\xcf\x84\xd7D\xb1+\x83\xb6KQ\x1f~',2005-05-16 16:33:33
4,5,b'\x00',Sra.,Pilar,None,Ackerman,None,pilar1@adventure-works.com,0,1 (11) 500 555-0132,F57E03FEA2FD0F74684C20758110CC7860F67523,/RPjvXw=,"<AdditionalContactInfo xmlns=""http://schemas.m...",b'\xab\xb8\x1f\xdf##0C\x9a\xb8T\xe1<\xe6\xd8\xf9',2005-05-16 16:33:33


In [9]:
# Removing unnecessary columns from the table
columns = ['NameStyle', 'Title', 'MiddleName', 'Suffix', 'EmailPromotion', 'PasswordHash', 'PasswordSalt', 'AdditionalContactInfo', 'rowguid', 'ModifiedDate']
df_contact.drop(columns, axis=1, inplace=True)
df_contact.head()

,ContactID,FirstName,LastName,EmailAddress,Phone
0,1,Gustavo,Achong,gustavo0@adventure-works.com,398-555-0132
1,2,Catherine,Abel,catherine0@adventure-works.com,747-555-0171
2,3,Kim,Abercrombie,kim2@adventure-works.com,334-555-0137
3,4,Humberto,Acevedo,humberto0@adventure-works.com,599-555-0127
4,5,Pilar,Ackerman,pilar1@adventure-works.com,1 (11) 500 555-0132


In [10]:
# Merging 'employees' and 'contact' on 'ContactID'
df_employees = pd.merge(df_employees, df_contact, on='ContactID', how='inner')
df_employees.drop(['ContactID'], axis=1, inplace=True)
columns = ['NationalIDNumber','LoginID','ManagerID','BirthDate','rowguid']
df_employees.drop(columns, axis=1, inplace=True)

In [11]:
# Lookup operations to replace HireDate and ModifiedDate as hire_date_key and modified_date_key
sql_dim_date = "SELECT date_key, full_date FROM adventureworks_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_date)

df_dim_hire_date = df_dim_date.rename(columns={"date_key" : "hire_date_key", "full_date" : "HireDate"})
df_employees.HireDate = df_employees.HireDate.astype('datetime64[ns]').dt.date
df_employees = pd.merge(df_employees, df_dim_hire_date, on='HireDate', how='left')
df_employees.drop(columns=['HireDate'], inplace=True)

df_dim_modified_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})
df_employees.ModifiedDate = df_employees.ModifiedDate.astype('datetime64[ns]').dt.date
df_employees = pd.merge(df_employees, df_dim_modified_date, on='ModifiedDate', how='left')
df_employees.drop(columns=['ModifiedDate'], inplace=True)
df_employees.head()

,EmployeeID,Title,MaritalStatus,Gender,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,FirstName,LastName,EmailAddress,Phone,hire_date_key,modified_date_key
0,1,Production Technician - WC60,M,M,0,21,30,1,Guy,Gilbert,guy1@adventure-works.com,320-555-0195,19960731,20040731
1,2,Marketing Assistant,S,M,0,42,41,1,Kevin,Brown,kevin0@adventure-works.com,150-555-0189,19970226,20040731
2,3,Engineering Manager,M,M,1,2,21,1,Roberto,Tamburello,roberto0@adventure-works.com,212-555-0187,19971212,20040731
3,4,Senior Tool Designer,S,M,0,48,80,1,Rob,Walters,rob0@adventure-works.com,612-555-0100,19980105,20040731
4,5,Tool Designer,M,M,0,9,24,1,Thierry,D'Hers,thierry0@adventure-works.com,168-555-0183,19980111,20040731


In [12]:
# Inserting primary key
df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))
df_employees.head()

,employee_key,EmployeeID,Title,MaritalStatus,Gender,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,FirstName,LastName,EmailAddress,Phone,hire_date_key,modified_date_key
0,1,1,Production Technician - WC60,M,M,0,21,30,1,Guy,Gilbert,guy1@adventure-works.com,320-555-0195,19960731,20040731
1,2,2,Marketing Assistant,S,M,0,42,41,1,Kevin,Brown,kevin0@adventure-works.com,150-555-0189,19970226,20040731
2,3,3,Engineering Manager,M,M,1,2,21,1,Roberto,Tamburello,roberto0@adventure-works.com,212-555-0187,19971212,20040731
3,4,4,Senior Tool Designer,S,M,0,48,80,1,Rob,Walters,rob0@adventure-works.com,612-555-0100,19980105,20040731
4,5,5,Tool Designer,M,M,0,9,24,1,Thierry,D'Hers,thierry0@adventure-works.com,168-555-0183,19980111,20040731


In [13]:
# Writing dim_employees to adventureworks_dw 
table_name = "dim_employees"
primary_key = "employee_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_employees, table_name, primary_key, db_operation)

### 2.2 Creating Product Dimension Table
- I am extracting data from the 'product' table from the MySQL database to create the product dimension table, 'dim_products'

In [14]:
# Extracting data from 'product' table
sql_products = "SELECT * FROM adventureworks.product;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xb7\x15Bi\xf7\x08\rL\xac\xb1\xd74\xbaD\xc0\...,2004-03-11 10:01:36
1,2,Bearing Ball,BA-8327,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b' <\xaeX:OIG\xa7\xd4\xd5h\x80l\xc57',2004-03-11 10:01:36
2,3,BB Ball Bearing,BE-2349,b'\x01',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xd2\xae!\x9c\xfa[\x18O\xbc\xb8\xf1\x168\xdc.N',2004-03-11 10:01:36
3,4,Headset Ball Bearings,BE-2908,b'\x00',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'\xcb\xd6\xfe\xec\xffQ\xb5I\xb0l}\x8a\xc84\xd...,2004-03-11 10:01:36
4,316,Blade,BL-2036,b'\x01',b'\x00',None,800,600,0.0,0.0,...,None,None,None,NaN,NaN,1998-06-01,NaT,None,b'P\x97>\xe7;`1A\x89\xf5=\xd1^\xd5\xff\x80',2004-03-11 10:01:36


In [15]:
# Cleaning the flag codes
df_products['MakeFlag'] = df_products['MakeFlag'].str[-1]
df_products['FinishedGoodsFlag'] = df_products['FinishedGoodsFlag'].str[-1]

# Removing unnecessary columns from the table
columns = ['Color', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'ProductLine', 'Class', 'Style', 'ProductSubcategoryID', 'ProductModelID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'rowguid']
df_products.drop(columns, axis=1, inplace=True)
df_products.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,DaysToManufacture,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,1000,750,0.0,0.0,0,2004-03-11 10:01:36
1,2,Bearing Ball,BA-8327,0,0,1000,750,0.0,0.0,0,2004-03-11 10:01:36
2,3,BB Ball Bearing,BE-2349,1,0,800,600,0.0,0.0,1,2004-03-11 10:01:36
3,4,Headset Ball Bearings,BE-2908,0,0,800,600,0.0,0.0,0,2004-03-11 10:01:36
4,316,Blade,BL-2036,1,0,800,600,0.0,0.0,1,2004-03-11 10:01:36


In [16]:
# Lookup operation to replace ModifiedDate as modified_date_key
df_products.ModifiedDate = df_products.ModifiedDate.astype('datetime64[ns]').dt.date
df_products = pd.merge(df_products, df_dim_modified_date, on='ModifiedDate', how='left')
df_products.drop(columns=['ModifiedDate'], inplace=True)

In [17]:
# Inserting primary key
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head()

,product_key,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,DaysToManufacture,modified_date_key
0,1,1,Adjustable Race,AR-5381,0,0,1000,750,0.0,0.0,0,20040311
1,2,2,Bearing Ball,BA-8327,0,0,1000,750,0.0,0.0,0,20040311
2,3,3,BB Ball Bearing,BE-2349,1,0,800,600,0.0,0.0,1,20040311
3,4,4,Headset Ball Bearings,BE-2908,0,0,800,600,0.0,0.0,0,20040311
4,5,316,Blade,BL-2036,1,0,800,600,0.0,0.0,1,20040311


In [18]:
# Writing dim_products to adventureworks_dw
table_name = "dim_products"
primary_key = "product_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_products, table_name, primary_key, db_operation)

### 2.3 Creating Vendor Dimension Table From CSV File
- First, I exported the data from the adventureworks.vendor table into a CSV file
- Below, I am loading the CSV data into the data warehouse

In [19]:
data_dir = os.path.join(os.getcwd(), 'data/adventureworks_vendor.csv')
df_vendor = pd.read_csv(data_dir)
df_vendor.head()

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1,INTERNAT0001,International,1,1,1,NaN,2002-02-25 00:00:00
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,NaN,2002-02-17 00:00:00
2,3,PREMIER0001,"Premier Sport, Inc.",1,1,1,NaN,2002-03-05 00:00:00
3,4,COMFORT0001,Comfort Road Bicycles,1,1,1,NaN,2002-01-24 00:00:00
4,5,METROSP0001,Metro Sport Equipment,1,1,1,NaN,2002-03-01 00:00:00


In [20]:
# Removing 'PurchasingWebServiceURL' from the table
columns = ['PurchasingWebServiceURL']
df_vendor.drop(columns, axis=1, inplace=True)

In [21]:
# Lookup operations to replace ModifiedDate as modified_date_key
df_vendor.ModifiedDate = df_vendor.ModifiedDate.astype('datetime64[ns]').dt.date
df_vendor = pd.merge(df_vendor, df_dim_modified_date, on='ModifiedDate', how='left')
df_vendor.drop(columns=['ModifiedDate'], inplace=True)

In [22]:
# Inserting primary key
df_vendor.insert(0, "vendor_key", range(1, df_vendor.shape[0]+1))
df_vendor.head()

,vendor_key,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,modified_date_key
0,1,1,INTERNAT0001,International,1,1,1,20020225
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,20020217
2,3,3,PREMIER0001,"Premier Sport, Inc.",1,1,1,20020305
3,4,4,COMFORT0001,Comfort Road Bicycles,1,1,1,20020124
4,5,5,METROSP0001,Metro Sport Equipment,1,1,1,20020301


In [23]:
# Writing dim_vendor to adventureworks_dw
table_name = "dim_vendor"
primary_key = "vendor_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_vendor, table_name, primary_key, db_operation)

### 2.4 Creating Shipmethod Dimension Table From JSON File
- First, I exported the data from the adventureworks.shipmethod table into a JSON file
- Below, I am loading the JSON data into the data warehouse

In [24]:
# Populating MongoDB with JSON data
client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')
json_files = {"shipmethod" : 'adventureworks_shipmethod.json'}
set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)

In [25]:
# Extracting data from MongoDB collection
client = get_mongo_client(**mongodb_args)
query = {}
collection = "shipmethod"

df_shipmethod = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_shipmethod.head()

,ShipMethodID,Name,ShipBase,ShipRate,rowguid,ModifiedDate
0,1,XRQ - TRUCK GROUND,3.95,0.99,None,1998-06-01 00:00:00
1,2,ZY - EXPRESS,9.95,1.99,None,1998-06-01 00:00:00
2,3,OVERSEAS - DELUXE,29.95,2.99,None,1998-06-01 00:00:00
3,4,OVERNIGHT J-FAST,21.95,1.29,None,1998-06-01 00:00:00
4,5,CARGO TRANSPORT 5,8.99,1.49,None,1998-06-01 00:00:00


In [26]:
# # Removing 'rowguid' from the table
columns = ['rowguid']
df_shipmethod.drop(columns, axis=1, inplace=True)

In [27]:
# Lookup operation to replace ModifiedDate as modified_date_key
df_shipmethod.ModifiedDate = df_shipmethod.ModifiedDate.astype('datetime64[ns]').dt.date
df_shipmethod = pd.merge(df_shipmethod, df_dim_modified_date, on='ModifiedDate', how='left')
df_shipmethod.drop(columns=['ModifiedDate'], inplace=True)

In [28]:
# Inserting primary key 
df_shipmethod.insert(0, "shipmethod_key", range(1, df_shipmethod.shape[0]+1))
df_shipmethod.head()

,shipmethod_key,ShipMethodID,Name,ShipBase,ShipRate,modified_date_key
0,1,1,XRQ - TRUCK GROUND,3.95,0.99,19980601
1,2,2,ZY - EXPRESS,9.95,1.99,19980601
2,3,3,OVERSEAS - DELUXE,29.95,2.99,19980601
3,4,4,OVERNIGHT J-FAST,21.95,1.29,19980601
4,5,5,CARGO TRANSPORT 5,8.99,1.49,19980601


In [29]:
# Writing dim_shipmethod to adventureworks_dw
table_name = "dim_shipmethod"
primary_key = "shipmethod_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_shipmethod, table_name, primary_key, db_operation)

## Creating and Populating the Fact Table: Purchase Orders

In [30]:
# Extracting data from 'purchaseorderheader' table from the source database
purchase_order_header = "SELECT * FROM adventureworks.purchaseorderheader;"
df_fact_po = get_dataframe(user_id, pwd, host_name, dst_dbname, purchase_order_header)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,0,4,244,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26
1,2,0,1,231,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26
2,3,0,4,241,38,2,2001-05-17,2001-05-26,8847.3000,707.7840,221.1825,9776.2665,2001-05-26
3,4,0,3,266,85,5,2001-05-17,2001-05-26,171.0765,13.6861,4.2769,189.0395,2001-05-26
4,5,0,4,164,92,4,2001-05-31,2001-06-09,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09


### 3.0 Importing Surrogate and Primary Keys For Each Dimension

In [31]:
# dim_employees
sql_dim_employees = "SELECT employee_key, EmployeeID FROM adventureworks_dw.dim_employees;"
df_dim_employees = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_employees)
df_dim_employees.head()

,employee_key,EmployeeID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [32]:
# dim_vendor
sql_dim_vendor = "SELECT vendor_key, VendorID FROM adventureworks_dw.dim_vendor;"
df_dim_vendor = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_vendor)
df_dim_vendor.head()

,vendor_key,VendorID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [33]:
# dim_shipmethod
sql_dim_shipmethod = "SELECT shipmethod_key, ShipMethodID FROM adventureworks_dw.dim_shipmethod;"
df_dim_shipmethod = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_shipmethod)
df_dim_shipmethod.head()

,shipmethod_key,ShipMethodID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


### 3.1 Lookup Surrogate Primary Keys

In [34]:
# Merging df_fact_po and dim_employees
# Retrieving 'employee_key' and dropping 'EmployeeId' column 
df_fact_po = pd.merge(df_fact_po, df_dim_employees, on='EmployeeID', how='left')
df_fact_po.drop(columns=['EmployeeID'],inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key
0,1,0,4,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244
1,2,0,1,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231
2,3,0,4,38,2,2001-05-17,2001-05-26,8847.3000,707.7840,221.1825,9776.2665,2001-05-26,241
3,4,0,3,85,5,2001-05-17,2001-05-26,171.0765,13.6861,4.2769,189.0395,2001-05-26,266
4,5,0,4,92,4,2001-05-31,2001-06-09,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09,164


In [35]:
# Merging df_fact_po and dim_vendors
# Retrieving 'vendor_key' and dropping 'VendorId' column 
df_fact_po = pd.merge(df_fact_po, df_dim_vendor, on='VendorID', how='left')
df_fact_po.drop(columns=['VendorID'],inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key
0,1,0,4,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83
1,2,0,1,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32
2,3,0,4,2,2001-05-17,2001-05-26,8847.3000,707.7840,221.1825,9776.2665,2001-05-26,241,38
3,4,0,3,5,2001-05-17,2001-05-26,171.0765,13.6861,4.2769,189.0395,2001-05-26,266,85
4,5,0,4,4,2001-05-31,2001-06-09,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09,164,92


In [36]:
# Merging df_fact_po and dim_shipmethod
# Retrieving 'shipmethod_key' and drop 'ShipMethodId' column 
df_fact_po = pd.merge(df_fact_po, df_dim_shipmethod, on='ShipMethodID', how='left')
df_fact_po.drop(columns=['ShipMethodID'],inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key
0,1,0,4,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3
1,2,0,1,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5
2,3,0,4,2001-05-17,2001-05-26,8847.3000,707.7840,221.1825,9776.2665,2001-05-26,241,38,2
3,4,0,3,2001-05-17,2001-05-26,171.0765,13.6861,4.2769,189.0395,2001-05-26,266,85,5
4,5,0,4,2001-05-31,2001-06-09,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09,164,92,4


### 3.2 Lookup Date Keys

In [37]:
# Order Date
df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "OrderDate"})
df_fact_po.OrderDate = df_fact_po.OrderDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_order_date, on='OrderDate', how='left')
df_fact_po.drop(columns=['OrderDate'], inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key,order_date_key
0,1,0,4,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3,20010517
1,2,0,1,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5,20010517
2,3,0,4,2001-05-26,8847.3000,707.7840,221.1825,9776.2665,2001-05-26,241,38,2,20010517
3,4,0,3,2001-05-26,171.0765,13.6861,4.2769,189.0395,2001-05-26,266,85,5,20010517
4,5,0,4,2001-06-09,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09,164,92,4,20010531


In [38]:
# Ship Date 
df_dim_ship_date = df_dim_date.rename(columns={"date_key" : "ship_date_key", "full_date" : "ShipDate"})
df_fact_po.ShipDate = df_fact_po.ShipDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_ship_date, on='ShipDate', how='left')
df_fact_po.drop(columns=['ShipDate'], inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key
0,1,0,4,201.0400,16.0832,5.0260,222.1492,2001-05-26,244,83,3,20010517,20010526
1,2,0,1,272.1015,21.7681,6.8025,300.6721,2001-05-26,231,32,5,20010517,20010526
2,3,0,4,8847.3000,707.7840,221.1825,9776.2665,2001-05-26,241,38,2,20010517,20010526
3,4,0,3,171.0765,13.6861,4.2769,189.0395,2001-05-26,266,85,5,20010517,20010526
4,5,0,4,20397.3000,1631.7840,509.9325,22539.0165,2001-06-09,164,92,4,20010531,20010609


In [39]:
# Modified Date
df_dim_modified_date = df_dim_date.rename(columns={"date_key" : "modified_date_key", "full_date" : "ModifiedDate"})
df_fact_po.ModifiedDate = df_fact_po.ModifiedDate.astype('datetime64[ns]').dt.date
df_fact_po = pd.merge(df_fact_po, df_dim_modified_date, on='ModifiedDate', how='left')
df_fact_po.drop(columns=['ModifiedDate'], inplace=True)
df_fact_po.head()

,PurchaseOrderID,RevisionNumber,Status,SubTotal,TaxAmt,Freight,TotalDue,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key,modified_date_key
0,1,0,4,201.0400,16.0832,5.0260,222.1492,244,83,3,20010517,20010526,20010526
1,2,0,1,272.1015,21.7681,6.8025,300.6721,231,32,5,20010517,20010526,20010526
2,3,0,4,8847.3000,707.7840,221.1825,9776.2665,241,38,2,20010517,20010526,20010526
3,4,0,3,171.0765,13.6861,4.2769,189.0395,266,85,5,20010517,20010526,20010526
4,5,0,4,20397.3000,1631.7840,509.9325,22539.0165,164,92,4,20010531,20010609,20010609


In [40]:
# Inserting primary key 
df_fact_po.insert(0, "purchase_order_key", range(1, df_fact_po.shape[0]+1))
df_fact_po.head()

,purchase_order_key,PurchaseOrderID,RevisionNumber,Status,SubTotal,TaxAmt,Freight,TotalDue,employee_key,vendor_key,shipmethod_key,order_date_key,ship_date_key,modified_date_key
0,1,1,0,4,201.0400,16.0832,5.0260,222.1492,244,83,3,20010517,20010526,20010526
1,2,2,0,1,272.1015,21.7681,6.8025,300.6721,231,32,5,20010517,20010526,20010526
2,3,3,0,4,8847.3000,707.7840,221.1825,9776.2665,241,38,2,20010517,20010526,20010526
3,4,4,0,3,171.0765,13.6861,4.2769,189.0395,266,85,5,20010517,20010526,20010526
4,5,5,0,4,20397.3000,1631.7840,509.9325,22539.0165,164,92,4,20010531,20010609,20010609


In [41]:
# Writing fact_purchase_orders to adventureworks_dw
table_name = "fact_purchase_orders"
primary_key = "purchase_order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_po, table_name, primary_key, db_operation)

## Authoring Summarization Queries

### 4.0 Average Total of Purchase Orders by Vendor 

In [42]:
sql_purchase_orders = """SELECT v.Name as vendor, AVG(po.TotalDue) as average_total_due
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_vendor as v
ON po.vendor_key = v.vendor_key
GROUP BY v.Name;
"""
df_q1 = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_purchase_orders)
df_q1.head()

,vendor,average_total_due
0,"Litware, Inc.",167.712239
1,Advanced Bicycles,558.864545
2,Allenson Cycles,9776.266500
3,American Bicycles and Wheels,189.039500
4,American Bikes,22539.016500


### 4.1 Overall Subtotal For Each Employee

In [43]:
sql_employee_totals = """SELECT concat(e.FirstName, ' ', e.LastName) as employee, Sum(po.SubTotal) as overall_subtotal
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_employees as e
ON po.employee_key = e.employee_key
GROUP BY employee;
"""
df_q2 = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_employee_totals)
df_q2.head()

,employee,overall_subtotal
0,Erin Hagens,4.633367e+06
1,Fukiko Ogisu,4.906001e+06
2,Eric Kurjan,6.283091e+06
3,Reinout Hillmann,6.536732e+06
4,Mikael Sandberg,6.169401e+06


### 4.2 Average Freight Cost by Shipmethod

In [44]:
sql_avg_freight_2002 = """SELECT s.Name as ship_method, AVG(po.Freight) as average_freight_cost
FROM adventureworks_dw.fact_purchase_orders as po
LEFT OUTER JOIN adventureworks_dw.dim_shipmethod as s
ON po.shipmethod_key = s.shipmethod_key
WHERE po.ship_date_key BETWEEN 20020101 AND 20021231 
GROUP BY s.Name;
"""
df_q3 = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_avg_freight_2002)
df_q3.head()

,ship_method,average_freight_cost
0,CARGO TRANSPORT 5,475.224583
1,OVERNIGHT J-FAST,246.043712
2,ZY - EXPRESS,555.085007
3,XRQ - TRUCK GROUND,128.327267
4,OVERSEAS - DELUXE,819.872400
